In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from torch_geometric.data import download_url, extract_zip
import os
import pandas as pd

In [3]:
# url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
url = 'https://files.grouplens.org/datasets/movielens/ml-latest.zip'
extract_zip(download_url(url, '.'), '.')

Using existing file ml-latest.zip
Extracting .\ml-latest.zip


In [4]:
os.listdir('./ml-latest')

['.ipynb_checkpoints',
 'genome-scores.csv',
 'genome-tags.csv',
 'links.csv',
 'movies.csv',
 'ratings.csv',
 'README.txt',
 'tags.csv']

In [5]:
movies_df = pd.read_csv('./ml-latest/movies.csv')
ratings_df = pd.read_csv('./ml-latest/ratings.csv')

In [6]:
movies_df.shape
movies_df.head()

(86537, 3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
movies_df.set_index('movieId', inplace=True)

In [9]:
genres = movies_df['genres'].str.get_dummies("|")

In [10]:
import torch

In [11]:
movie_features = torch.from_numpy(genres.values).to(torch.float)

In [12]:
genres.shape
movie_features.shape
movie_features
genres.head()

(86537, 20)

torch.Size([86537, 20])

tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
ratings_df.shape
ratings_df.head()
# we will consider movie ratings above 3 for recommending

(33832162, 4)

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


In [14]:
ratings_df['rating'].value_counts()

rating
4.0    8835955
3.0    6400664
5.0    4957902
3.5    4465001
4.5    3123055
2.0    2146492
2.5    1760733
1.0    1013645
0.5     566306
1.5     562409
Name: count, dtype: int64

In [15]:
len(ratings_df['userId'].unique())
len(ratings_df[ratings_df['rating']>=3]['userId'].unique())

len(ratings_df['userId'].unique()) - len(ratings_df[ratings_df['rating']>=3.5]['userId'].unique())

330975

329127

4295

In [16]:
ratings_df = ratings_df[ratings_df['rating'] >= 3.0]

In [17]:
ratings_df.shape
ratings_df['rating'].value_counts()

(27782577, 4)

rating
4.0    8835955
3.0    6400664
5.0    4957902
3.5    4465001
4.5    3123055
Name: count, dtype: int64

## Create Edge Index

#### Create user id edge index

In [21]:
unique_user_id = ratings_df['userId'].unique()
unique_user_id = pd.DataFrame(data = {'userId':unique_user_id, 'mappedID':pd.RangeIndex(start=0, stop=len(unique_user_id))})

In [26]:
unique_user_id.head(2)
ratings_df.head(2)

,userId,mappedID
0,1,0
1,2,1


,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086


In [27]:
user_id_edge_index = pd.merge(ratings_df, unique_user_id, left_on= 'userId', right_on='userId', how='left')['mappedID'].values

In [28]:
user_id_edge_index = torch.from_numpy(user_id_edge_index)

In [29]:
user_id_edge_index

tensor([     0,      0,      0,  ..., 329126, 329126, 329126])

#### Create movie id edge index

In [30]:
unique_movie_id = pd.DataFrame(data = {'movieId': movies_df.index, 'mappedID': pd.RangeIndex(start=0, stop=movies_df.shape[0])})

In [31]:
unique_movie_id.head(2)
ratings_df.head(2)

,movieId,mappedID
0,1,0
1,2,1


,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086


In [32]:
movie_id_edge_index = pd.merge(ratings_df, unique_movie_id, left_on='movieId', right_on='movieId', how='left')['mappedID'].values

In [33]:
movie_id_edge_index

array([   0,  108,  156, ..., 7029, 7911, 7954], dtype=int64)

In [21]:
movie_id_edge_index = torch.from_numpy(movie_id_edge_index)

#### Concatenate to create user-movie edge index

In [22]:
user_id_edge_index
movie_id_edge_index

user_id_edge_index.shape
movie_id_edge_index.shape

torch.stack([user_id_edge_index, movie_id_edge_index], dim=0).shape
torch.stack([user_id_edge_index, movie_id_edge_index], dim=0)

tensor([     0,      0,      0,  ..., 198445, 198445, 198445])

tensor([    0,   108,   156,  ..., 11985, 12029, 12113])

torch.Size([16798323])

torch.Size([16798323])

torch.Size([2, 16798323])

tensor([[     0,      0,      0,  ..., 198445, 198445, 198445],
        [     0,    108,    156,  ...,  11985,  12029,  12113]])

In [23]:
edge_index_user2movie = torch.stack([user_id_edge_index, movie_id_edge_index], dim=0)
edge_index_user2movie.shape
edge_index_user2movie

torch.Size([2, 16798323])

tensor([[     0,      0,      0,  ..., 198445, 198445, 198445],
        [     0,    108,    156,  ...,  11985,  12029,  12113]])

### Creation of heterogenous graph data

In [24]:
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

In [34]:
movie_features

tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [25]:
data = HeteroData()

data['user']['node_id'] = torch.arange(len(unique_user_id))
data['movie']['node_id'] = torch.arange(len(movies_df))

data['movie']['x'] = movie_features
data['user', 'rates', 'movie']['edge_index'] = edge_index_user2movie

data = T.ToUndirected()(data)

In [26]:
data

HeteroData(
  user={ node_id=[198446] },
  movie={
    node_id=[86537],
    x=[86537, 20],
  },
  (user, rates, movie)={ edge_index=[2, 16798323] },
  (movie, rev_rates, user)={ edge_index=[2, 16798323] }
)

In [28]:
data.node_types
data.edge_types
data['user'].num_nodes
data['movie'].num_nodes
data['movie'].x.shape # node features
data['user', 'rates', 'movie']['edge_index'].shape

['user', 'movie']

[('user', 'rates', 'movie'), ('movie', 'rev_rates', 'user')]

198446

86537

torch.Size([86537, 20])

torch.Size([2, 16798323])

### Define Edge level training Splits

In [29]:
# train :: 80% ; val :: 10% ; test :: 10%
# from the train we use 70% for message_passing and 30% for supervision (disjoint_train_ratio)
# generate negative edges for evaluation 2:1 ratio (generated on the fly)

transform = T.RandomLinkSplit(num_val= 0.1, num_test=0.1, disjoint_train_ratio=0.3, 
                              neg_sampling_ratio=2.0, add_negative_train_samples=False, 
                              edge_types=('user', 'rates', 'movie'), rev_edge_types=('movie', 'rev_rates', 'user'))

In [30]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [31]:
device

'cuda'

In [32]:
data.to(device)

HeteroData(
  user={ node_id=[198446] },
  movie={
    node_id=[86537],
    x=[86537, 20],
  },
  (user, rates, movie)={ edge_index=[2, 16798323] },
  (movie, rev_rates, user)={ edge_index=[2, 16798323] }
)

In [33]:
train_data, val_data, test_data = transform(data)

In [34]:
train_data
val_data
test_data

HeteroData(
  user={ node_id=[198446] },
  movie={
    node_id=[86537],
    x=[86537, 20],
  },
  (user, rates, movie)={
    edge_index=[2, 9407062],
    edge_label=[4031597],
    edge_label_index=[2, 4031597],
  },
  (movie, rev_rates, user)={ edge_index=[2, 9407062] }
)

HeteroData(
  user={ node_id=[198446] },
  movie={
    node_id=[86537],
    x=[86537, 20],
  },
  (user, rates, movie)={
    edge_index=[2, 13438659],
    edge_label=[5039496],
    edge_label_index=[2, 5039496],
  },
  (movie, rev_rates, user)={ edge_index=[2, 13438659] }
)

HeteroData(
  user={ node_id=[198446] },
  movie={
    node_id=[86537],
    x=[86537, 20],
  },
  (user, rates, movie)={
    edge_index=[2, 15118491],
    edge_label=[5039496],
    edge_label_index=[2, 5039496],
  },
  (movie, rev_rates, user)={ edge_index=[2, 15118491] }
)

In [35]:
train_data['user', 'rates', 'movie'].num_edges
val_data['user', 'rates', 'movie'].num_edges

train_data['movie', 'rev_rates', 'user'].num_edges
val_data['movie', 'rev_rates', 'user'].num_edges

9407062

13438659

9407062

13438659

In [36]:
train_data['user', 'rates', 'movie']['edge_label'].min()
train_data['user', 'rates', 'movie']['edge_label'].max()

#negative samples created in val data with the ratio 2:1
val_data['user', 'rates', 'movie']['edge_label'].min()
val_data['user', 'rates', 'movie']['edge_label'].max()
val_data['user', 'rates', 'movie']['edge_label'].long().bincount().tolist()

tensor(1., device='cuda:0')

tensor(1., device='cuda:0')

tensor(0., device='cuda:0')

tensor(1., device='cuda:0')

[3359664, 1679832]

### Get Mini Batch loaders

In [37]:
from torch_geometric.loader import LinkNeighborLoader

In [38]:
train_data

HeteroData(
  user={ node_id=[198446] },
  movie={
    node_id=[86537],
    x=[86537, 20],
  },
  (user, rates, movie)={
    edge_index=[2, 9407062],
    edge_label=[4031597],
    edge_label_index=[2, 4031597],
  },
  (movie, rev_rates, user)={ edge_index=[2, 9407062] }
)

In [40]:
edge_label_index = train_data['user', 'rates', 'movie']['edge_label_index']
edge_label = train_data['user', 'rates', 'movie']['edge_label']
train_loader = LinkNeighborLoader(data = train_data, 
                                  num_neighbors= [20,10], 
                                  neg_sampling_ratio= 2.0, 
                                  batch_size = 128, 
                                  shuffle = True,
                                  edge_label= edge_label,
                                  edge_label_index= (('user', 'rates', 'movie'), edge_label_index))

In [41]:
edge_label_index
edge_label

tensor([[130286, 161332,  50389,  ..., 161714,  63378,  32759],
        [ 12333,   2480,   3480,  ...,   3326,   1107,   2077]],
       device='cuda:0')

tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0')

In [42]:
sample_batch_data = next(iter(train_loader))

In [43]:
sample_batch_data['user', 'rates', 'movie']['edge_label_index'].size()
sample_batch_data['user', 'rates', 'movie']['edge_label'].long().bincount() #['0s count', '1s count'] because negative_sampling is 2:1

torch.Size([2, 384])

tensor([256, 128])

In [44]:
sample_batch_data

HeteroData(
  user={
    node_id=[19184],
    n_id=[19184],
  },
  movie={
    node_id=[7463],
    x=[7463, 20],
    n_id=[7463],
  },
  (user, rates, movie)={
    edge_index=[2, 26037],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[26037],
    input_id=[128],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 37312],
    e_id=[37312],
  }
)

### Creating a heterogeneous link-level GNN

In [46]:
from torch_geometric.nn import SAGEConv, to_hetero
import torch
import torch.nn.functional as F

In [48]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels= hidden_channels, out_channels=hidden_channels)
        self.conv2 = SAGEConv(in_channels= hidden_channels, out_channels=hidden_channels)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [ ]:
class Classifier(torch.nn.Module):
    def forward(self, x_user: Tensor, x_movie: Tensor, edge_label_index: Tensor) -> Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_user = x_user[edge_label_index[0]]
        edge_feat_movie = x_movie[edge_label_index[1]]
        # Apply dot-product to get a prediction per supervision edge:
        return (edge_feat_user * edge_feat_movie).sum(dim=-1)

In [49]:
class Classifier(torch.nn.Module):
    def forward(self, x_user, x_movie, edge_label_index):
        user_edge_feature = x_user[edge_label_index[0]]
        movie_edge_feature = x_movie[edge_label_index[1]]
        
        return (user_edge_feature * movie_edge_feature).sum(dim=-1)

In [153]:
data
data['user'].num_nodes
data['movie'].num_nodes

HeteroData(
  user={ node_id=[198446] },
  movie={
    node_id=[86537],
    x=[86537, 20],
  },
  edge_index_dict={},
  (user, rates, movie)={ edge_index=[2, 16798323] },
  (movie, rev_rates, user)={ edge_index=[2, 16798323] }
)

198446

86537

In [154]:
data.node_types
data.edge_types
data.metadata()

['user', 'movie', 'edge_index_dict']

[('user', 'rates', 'movie'), ('movie', 'rev_rates', 'user')]

(['user', 'movie', 'edge_index_dict'],
 [('user', 'rates', 'movie'), ('movie', 'rev_rates', 'user')])

In [155]:
data['user']['node_id']
data['movie']['x']

tensor([     0,      1,      2,  ..., 198443, 198444, 198445], device='cuda:0')

tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [156]:
data.edge_index_dict

{('user',
  'rates',
  'movie'): tensor([[     0,      0,      0,  ..., 198445, 198445, 198445],
         [     0,    108,    156,  ...,  11985,  12029,  12113]],
        device='cuda:0'),
 ('movie',
  'rev_rates',
  'user'): tensor([[     0,    108,    156,  ...,  11985,  12029,  12113],
         [     0,      0,      0,  ..., 198445, 198445, 198445]],
        device='cuda:0')}

In [50]:
class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        
        # user and movie embedding matrices
        self.movie_lin = torch.nn.Linear(20, hidden_channels)
        self.user_emb = torch.nn.Embedding(num_embeddings=data['user'].num_nodes, embedding_dim=hidden_channels)
        self.movie_emb = torch.nn.Embedding(num_embeddings=data['movie'].num_nodes, embedding_dim=hidden_channels)
        
        self.gnn = GNN(hidden_channels=hidden_channels)
        self.gnn = to_hetero(module=self.gnn, metadata=data.metadata())
        
        self.classifier = Classifier()
    
    def forward(self, data: HeteroData):
        x_dict = {"user" : self.user_emb(data['user'].node_id), 
                  "movie": self.movie_lin(data['movie'].x) + self.movie_emb(data['movie'].node_id)
                 }
        x_dict = self.gnn(x_dict, data.edge_index_dict)
        
        pred = self.classifier(x_dict['user'], 
                               x_dict['movie'], 
                               data['user', 'rates', 'movie'].edge_label_index)
        
        return pred
    

In [51]:
model = Model(hidden_channels=64)

C:\Users\Sridhar Kamoji\Python\GNN\.env-gnn\lib\site-packages\torch\overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
C:\Users\Sridhar Kamoji\Python\GNN\.env-gnn\lib\site-packages\torch\overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
C:\Users\Sridhar Kamoji\Python\GNN\.env-gnn\lib\site-packages\torch\overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
C:\Users\Sridhar Kamoji\Python\GNN\.env-gnn\lib\site-packages\torch\overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


### training

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [54]:
device

device(type='cuda')

In [55]:
model = model.to(device)

In [56]:
model

Model(
  (movie_lin): Linear(in_features=20, out_features=64, bias=True)
  (user_emb): Embedding(198446, 64)
  (movie_emb): Embedding(86537, 64)
  (gnn): GraphModule(
    (conv1): ModuleDict(
      (user__rates__movie): SAGEConv(64, 64, aggr=mean)
      (movie__rev_rates__user): SAGEConv(64, 64, aggr=mean)
    )
    (conv2): ModuleDict(
      (user__rates__movie): SAGEConv(64, 64, aggr=mean)
      (movie__rev_rates__user): SAGEConv(64, 64, aggr=mean)
    )
  )
  (classifier): Classifier()
)

In [57]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [58]:
import tqdm

In [59]:
sample_batch_data

HeteroData(
  user={
    node_id=[19184],
    n_id=[19184],
  },
  movie={
    node_id=[7463],
    x=[7463, 20],
    n_id=[7463],
  },
  (user, rates, movie)={
    edge_index=[2, 26037],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[26037],
    input_id=[128],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 37312],
    e_id=[37312],
  }
)

In [ ]:
for epoch in range(1,20):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()
        sampled_data.to(device)
        pred = model(sampled_data)
        
        ground_truth = sampled_data['user', 'rates', 'movie']['edge_label']
        loss = F.binary_cross_entropy_with_logits(pred, ground_truth)
        loss.backward()
        
        optimizer.step()
        
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

  0%|                                                                                        | 0/31497 [00:00<?, ?it/s]

HeteroData(
  user={
    node_id=[19261],
    n_id=[19261],
  },
  movie={
    node_id=[7579],
    x=[7579, 20],
    n_id=[7579],
  },
  (user, rates, movie)={
    edge_index=[2, 26473],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[26473],
    input_id=[128],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 36823],
    e_id=[36823],
  }
)

  0%|                                                                              | 1/31497 [00:00<7:37:19,  1.15it/s]

HeteroData(
  user={
    node_id=[19535],
    n_id=[19535],
  },
  movie={
    node_id=[7665],
    x=[7665, 20],
    n_id=[7665],
  },
  (user, rates, movie)={
    edge_index=[2, 26919],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[26919],
    input_id=[128],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 38488],
    e_id=[38488],
  }
)

  0%|                                                                              | 2/31497 [00:01<4:38:01,  1.89it/s]

...

HeteroData(
  user={
    node_id=[18625],
    n_id=[18625],
  },
  movie={
    node_id=[7516],
    x=[7516, 20],
    n_id=[7516],
  },
  (user, rates, movie)={
    edge_index=[2, 25205],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[25205],
    input_id=[128],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 36668],
    e_id=[36668],
  }
)

  0%|▎                                                                           | 142/31497 [00:32<2:20:58,  3.71it/s]

HeteroData(
  user={
    node_id=[20056],
    n_id=[20056],
  },
  movie={
    node_id=[7959],
    x=[7959, 20],
    n_id=[7959],
  },
  (user, rates, movie)={
    edge_index=[2, 28115],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[28115],
    input_id=[128],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 39115],
    e_id=[39115],
  }
)

  0%|▎                                                                           | 143/31497 [00:32<2:05:06,  4.18it/s]

HeteroData(
  user={
    node_id=[19088],
    n_id=[19088],
  },
  movie={
    node_id=[7719],
    x=[7719, 20],
    n_id=[7719],
  },
  (user, rates, movie)={
    edge_index=[2, 26206],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[26206],
    input_id=[128],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 39270],
    e_id=[39270],
  }
)